In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image
import PIL



In [2]:
# Define paths
train_dir = 'Data/Img'

# Parameters
batch_size = 32
img_height = 224
img_width = 224
epochs = 10

In [3]:
# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of the data for validation
)


In [4]:
# Training and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)


Found 275 images belonging to 2 classes.


In [5]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 67 images belonging to 2 classes.


In [6]:
# Load the pre-trained MobileNetV2 model + higher level layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


In [7]:
# Freeze the base model
base_model.trainable = False

In [8]:
# Add new layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [9]:
model = Model(inputs=base_model.input, outputs=predictions)


In [10]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [11]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)


In [12]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/10


/Users/rohan/Developer/Projects/textpredict/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 436ms/step - accuracy: 0.7048 - loss: 0.5322 - val_accuracy: 0.6250 - val_loss: 0.5296
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8438 - loss: 0.3135 - val_accuracy: 1.0000 - val_loss: 0.0269
Epoch 3/10


2024-07-16 23:54:54.828546: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/rohan/.pyenv/versions/3.11.9/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-07-16 23:54:54.861508: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 287ms/step - accuracy: 0.9365 - loss: 0.1682 - val_accuracy: 0.7812 - val_loss: 0.3574
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0739 - val_accuracy: 0.6667 - val_loss: 0.3554
Epoch 5/10


2024-07-16 23:55:00.852028: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 259ms/step - accuracy: 0.9543 - loss: 0.0904 - val_accuracy: 0.7969 - val_loss: 0.3905


In [13]:
# Unfreeze the base model
base_model.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])


In [14]:
# Continue training (fine-tuning)
fine_tune_epochs = 10
total_epochs = epochs + fine_tune_epochs

In [16]:
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 978ms/step - accuracy: 0.5494 - loss: 0.6573 - val_accuracy: 0.9062 - val_loss: 0.3031
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4375 - loss: 0.7522 - val_accuracy: 1.0000 - val_loss: 0.3271
Epoch 7/20


2024-07-16 23:57:42.965040: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 923ms/step - accuracy: 0.7573 - loss: 0.4490 - val_accuracy: 0.9688 - val_loss: 0.2862


In [17]:
# Save the final model
model.save('final_model.keras')

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('final_model.keras')

def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize
    return img_array

def predict_image(model, img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    return predictions

def interpret_prediction(predictions, threshold=0.5):
    if predictions[0] > threshold:
        return 'unrelevant'
    else:
        return 'relevant'

In [31]:
# Example usage
img_path = '/Users/rohan/Developer/Projects/textpredict/chatapp/static/uploads/Screenshot_2024-04-10_at_15.32.47.png'
predictions = predict_image(model, img_path)
class_prediction = interpret_prediction(predictions)
print(f'Predicted class: {class_prediction}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted class: relevant
